In [1]:
import numpy as np
import avapi
import avstack

Cannot import rss library -- don't worry about this unless you need 'safety' evals
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
nusc_data_dir = '/data/nuScenes'
NSM = avapi.nuscenes.nuScenesManager(nusc_data_dir)
NSD = NSM.get_scene_dataset_by_name('scene-0103')

## Test Visualization

In [6]:
frame = 20
img = NSD.get_image(frame, 'main_camera')
pc = NSD.get_lidar(frame, 'main_lidar')
objects = NSD.get_objects(frame, 'main_lidar')

# -- 2d camera
avapi.visualize.snapshot.show_image_with_boxes(img, objects, inline=True)

# -- 2d bev
vectors = [obj.velocity_head_tail for obj in objects]
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

KeyError: 'speed'

## Test Object Tracking

In [ ]:
first_frame = 1
last_frame = 20
tracker = avstack.modules.tracking.tracker3d.BasicBoxTracker3D(check_reference=True)
for frame in NSD.frames[first_frame:last_frame]:
    ts = NSD.get_timestamp(frame)
    ego = NSD.get_ego(frame)
    img = NSD.get_image(frame, sensor="main_camera")
    objects = NSD.get_objects(frame, sensor="main_camera")
    detections = [avstack.modules.perception.detections.BoxDetection(
                    'objects', obj.box3d, obj.reference, obj_type=obj.obj_type) for obj in objects]
    tracks = tracker(t=ts, frame=frame, detections=detections, platform=ego.as_reference())
    if frame > 1:
        avapi.visualize.snapshot.show_image_with_boxes(img, tracks, inline=True, show_IDs=True)

## Test Radar Tracking

In [ ]:
last_frame = 20
tracker = avstack.modules.tracking.tracker3d.BasicRazelTracker(threshold_confirmed=2, assign_radius=10)
for frame in NSD.frames[:last_frame]:
    ts = NSD.get_timestamp(frame)
    radar = NSD.get_radar(frame)
    ego = NSD.get_ego(frame)
    objects = NSD.get_objects(frame)
    img = NSD.get_image(frame, "main_camera")
    detections = [avstack.modules.perception.detections.RazelDetection(
                    radar.source_identifier, radar.data[i,:3], radar.reference)
                  for i in range(radar.data.shape[0])]
    detections = [det for det in detections if det.xyz[0] > 0]
    tracks = tracker(t=ts, frame=frame, detections=detections, platform=radar.reference)
    if frame > 1:
        avapi.visualize.snapshot.show_image_with_boxes(img, tracks, inline=True)

In [ ]:
frame = 6
pc = NSD.get_lidar(frame=frame)
pc.data = np.zeros((0,4))
objects = NSD.get_objects(frame=frame)
rad = NSD.get_radar(frame=frame)
det_xyz = avstack.geometry.transformations.matrix_spherical_to_cartesian(rad.data)

# -- vectors from detections
vectors = [obj.velocity_head_tail for obj in objects]
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

# -- vectors from tracks
# vectors = [obj.velocity_head_tail for obj in tracks]
# avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=tracks, vectors=vectors)